|      |      | 
| ---- | ---- | 
| Journal:|Sustainability (ISSN 2071-1050)|
|Manuscript ID:| sustainability-1485080|
|Type:|Article|
|Title:|NEVs Supply Chain Coordination with Financial Constraint and Demand Uncertainty|
|manuscripts:|P.6 - figure 3. Data generation code|
|Code authors:|Yongjian Li$^*$ |
|Last modify:|2021-01-12|
|version:|Python 3.6.5, sympy >= 1.3|

In [1]:
from sympy import *
from sympy.stats import *
import numpy as np
import pandas as pd
# init_printing(use_unicode=True)
import sympy
sympy.__version__

'1.9'

In [38]:
p, w, c, s, B, b, r, alpha, theta, q, a, x, r, mu, sigma, y = symbols(
    "p,w,c,s,B,b,r,alpha,theta,q,a,x,r,mu,sigma,y")

In [3]:
# Initial parameter values
parameter = {p: 10, c: 4, s: 3, w: 6, theta: 0.8, b: 3, B: 100, r: 0.05,
             mu: 100, sigma: 100}

(see manuscript P(6) figure 3.)

\begin{align}
&F^{-1}(\alpha) \\
&q_{t}^{d}=\frac{(\theta p-b) F^{-1}(\alpha)+B(1+r)}{w(1+r)-b} \\
&q_{s}^{d}=F^{-1}\left(\frac{\theta p-w(1+r)}{\theta p-b}\right)
\end{align}



- Suppose X is normally distributed with $\mu=100$,$\sigma=100$ 
- CDF,exp1: $F(x) = \alpha$

In [4]:
X = Normal(x, mu, sigma).subs(parameter)
exp1 = Eq(cdf(X)(x), alpha)
exp1

Eq(erf(sqrt(2)*(x - 100)/200)/2 + 1/2, alpha)

- inverse exp1: $F^{-1}(\alpha) = x ,\quad \{0\leqslant \alpha \leqslant 1\}$

In [5]:
i_cdf = solve(exp1, x)[0]
i_cdf

100*sqrt(2)*erfinv(2*alpha - 1) + 100

In [37]:
# Convert a SymPy expression into a function that allows for fast numeric evaluation.
f = lambdify(alpha, i_cdf, modules=['numpy', 'sympy'])

x_0 = [i/1000 for i in range(1, 1000)]

dt = {
    "x_0": x_0,
}

df = pd.DataFrame(dt)
df['f(x)'] = df['x_0'].map(f)
df['f(x)'] = df['f(x)'].map(int)
df1 = df.drop_duplicates(subset=['f(x)']).reset_index(drop=True).copy()
df1

,x_0,f(x)
0,0.001,-209
1,0.002,-187
2,0.003,-174
3,0.004,-165
4,0.005,-157
...,...,...
422,0.995,357
423,0.996,365
424,0.997,374
425,0.998,387


In [7]:
df1.query("`f(x)`==0")

,x_0,f(x)
113,0.157,0


In [8]:
df1.iloc[::10].to_csv("inverse(CDF)",index=None,sep='\t')

$$q^d_{s}=F^{-1}\left(\frac{\theta p-w(1+r)}{\theta p-b}\right)$$

In [30]:
prob = (p*theta - w*(1+r))/(theta*p - b)
prob

(p*theta - w*(r + 1))/(-b + p*theta)

In [31]:
prob =prob.subs(parameter).evalf(6)
prob

0.340000

In [32]:
eq = Eq(cdf(X)(a), prob)
qd_s = solve(eq, a)[0].subs(parameter)
qd_s = int(qd_s)
qd_s

58

$$q_{t}^{d}=\frac{(\theta p-b) F^{-1}(\alpha)+B(1+r)}{w(1+r)-b}$$

In [39]:
qd_t = ((p*theta-b) *x +B*(1+r))/(w*(1+r)-b)
qd_t

(B*(r + 1) + x*(-b + p*theta))/(-b + w*(r + 1))

In [45]:
qd_t = qd_t.subs(parameter)

f2 = lambdify(x, qd_t, modules=['numpy', 'sympy'])

qd_t_values = f2(df['f(x)'])
qd_t_values = qd_t_values.to_numpy()
qd_t_values.shape

(999,)

In [53]:
dt = {
    "x_0": x_0,
    "f(x)": qd_t_values
}
df_qd_t = pd.DataFrame(dt)
df_qd_t['f(x)'] = df_qd_t['f(x)'].map(int)
df_qd_t

,x_0,f(x)
0,0.001,-284
1,0.002,-251
2,0.003,-231
3,0.004,-218
4,0.005,-206
...,...,...
994,0.995,572
995,0.996,584
996,0.997,598
997,0.998,618


In [54]:
df_qd_t = df_qd_t.drop_duplicates(subset=['f(x)']).reset_index(drop=True)
df_qd_t = df_qd_t.iloc[::10].reset_index(drop=True)
df_qd_t

,x_0,f(x)
0,0.001,-284
1,0.011,-163
2,0.021,-124
3,0.031,-98
4,0.041,-78
5,0.051,-63
6,0.062,-48
7,0.075,-33
8,0.091,-18
9,0.108,-3


In [55]:
value = solve(Eq(qd_t, qd_s), x)[0]
d1 = {
    "x_0": [cdf(X)(value).evalf()],
    "f(x)": [qd_s]
}
df_qd_t = df_qd_t.append(pd.DataFrame(d1))


value = solve(Eq(qd_t, 0), x)[0]
d1 = {
    "x_0": [cdf(X)(value).evalf()],
    "f(x)": [0]
}

df_qd_t = df_qd_t.append(pd.DataFrame(d1))
df_qd_t = df_qd_t.drop_duplicates(subset=['x_0'])
df_qd_t = df_qd_t.sort_values(by=['x_0']).reset_index(drop=True)
df_qd_t

,x_0,f(x)
0,0.001,-284
1,0.011,-163
2,0.021,-124
3,0.031,-98
4,0.041,-78
5,0.051,-63
6,0.062,-48
7,0.075,-33
8,0.091,-18
9,0.108,-3


In [58]:
df_qd_t_e = df_qd_t.query("@qd_s >=`f(x)` >= 0").reset_index(drop=True)
df_qd_t_e


,x_0,f(x)
0,0.113139446443977,0
1,0.128,12
2,0.15,27
3,0.177,42
4,0.204,57
5,0.204061855829433,58


In [57]:
df_qd_t.to_csv("qd_t",index=None,sep='\t')
df_qd_t_e.to_csv("qd_t_e",index=None,sep='\t')

- some help fun() note here.

In [5]:
## note here
## Given a value, calculate the cumulative probability
value = 0
cdf(X)(value).evalf()
## Given a value, calculate the cumulative probability
value = 189.380
P(X < value).evalf(subs=parameter)